## Assessment of Influenza segment groupings

In order to ingest influenza segments in Loculus we must identify segments and group them, prior to creating this JSON this had to be done heuristically. Here I compare the results of using the heuristic approach (old) vs using the known (from assemblies) and heuristic approaches (new). 

Download all metadata using 

```
curl -X 'GET' \
  'https://api.loculus.staging.genspectrum.org/influenza-a/sample/details?dataFormat=JSON&downloadAsFile=true' \
  -H 'accept: application/json' > influenza_a_submissionIds.json
```

It is possible to just query LAPIS for the submissionIds of all sequences, this is what we need to compare groupings.

```
curl -X 'GET' \
  'https://api.loculus.staging.genspectrum.org/influenza-a/sample/details?fields=submissionId,AccessionVersion&dataFormat=JSON&downloadAsFile=true' \
  -H 'accept: application/json' > influenza_a_submissionIds.json
```

In numbers: 
1. There are a total of 1180715 influenza-a sequences on genbank.
2. Ingest uses nextclade sort to determine which influenza segment a sequence corresponds to, if a segment does not align to any segment it is dropped. Note I switch the `nextclade_sort` min score parameter from 0.1 to 0.05 as I was able to show it was correct for all segments where I had data for, I did not want to drop it further as I showed this led to inaccurate subtype assignments. Nextclade sort led to the dropping of 24144 sequences (over 4000 for 0.1).
3. Of these 346935 cannot be grouped using known assembly groupings.
4. Of the known assembly alignments there are 101247 known groups, there were 21 groups without all segments found (due to being previously dropped) and 3 full groups lost due to dropping a further 8 segment group was removed due to a submission error (submitter uploaded segment1 twice as segment1 and segment2). While grouping the known groups 4024 groups had ncbiReleaseDates that did not match, additionally 2890 had authors and 1731 had author affiliations that did not match. The number of fields where metadata fields are only found in some segments and not all is much higher, all of these would not be grouped by heuristic groupings.
5. Of the 346935 segments without a known grouping heuristic grouping can find 200747 groups.


In [50]:
import json

assembly_grouped = "influenza_a_submissionIds_new.json"
old_grouped = "influenza_a_submissionIds_old.json"
assembly_grouped_dict: dict = json.load(
    open(assembly_grouped, encoding="utf-8")
)
old_grouped_dict: dict = json.load(
    open(old_grouped, encoding="utf-8")
)

In [51]:
known_groups: dict = json.load(open("results/groups.json", encoding="utf-8"))
accession_to_true_group = {}
for group, accessions in known_groups.items():
    for accession in accessions:
        accession_to_true_group[accession] = set(accessions)

In [52]:
def parse_output(grouping_dict, grouped_accessions, groups, accession_to_group):
    sort_number_of_each_seg = {}
    for record in grouping_dict:
        insdc_accessions = record["submissionId"].split("/")
        grouped_accessions.extend(insdc_accessions)
        groups.append(record["submissionId"])
        insdc_accessions_no_segments = ['.'.join(record.split('.')[:-1]) for record in record["submissionId"].split("/")]
        for accession in insdc_accessions:
            seg = accession.split(".")[-1]
            accession_no_seg = '.'.join(accession.split(".")[:-1])
            sort_number_of_each_seg[seg] = sort_number_of_each_seg.get(seg, 0) + 1
            accession_to_group[accession_no_seg] = set(insdc_accessions_no_segments)

    print(f"Number of segments: {len(set(grouped_accessions))}")
    print(f"Number of groups: {len(set(groups))}")
    print(sort_number_of_each_seg)

assembly_grouped_accessions = []
assembly_groups = []
assembly_grouped_accessions_to_group = {}

parse_output(assembly_grouped_dict, assembly_grouped_accessions, assembly_groups, assembly_grouped_accessions_to_group)

Number of segments: 1156563
Number of groups: 301990
{'seg4': 214692, 'seg6': 162113, 'seg1': 126778, 'seg2': 124852, 'seg3': 126176, 'seg5': 128585, 'seg7': 143015, 'seg8': 130352}


In [53]:
old_grouped_accessions = []
old_groups = []
old_grouped_accessions_to_group = {}

parse_output(old_grouped_dict, old_grouped_accessions, old_groups, old_grouped_accessions_to_group)

Number of segments: 1135724
Number of groups: 364384
{'seg4': 194261, 'seg5': 128568, 'seg7': 143004, 'seg1': 126753, 'seg2': 124840, 'seg3': 126162, 'seg8': 130322, 'seg6': 161814}


In [54]:
print(f"Number of accessions only in new approach:{len(set(assembly_grouped_accessions) - set(old_grouped_accessions))}")
print(f"Number of accessions only in old approach:{len(set(old_grouped_accessions) - set(assembly_grouped_accessions))}")

print(f"Number of groups only in new approach:{len(set(assembly_groups) - set(old_groups))}")
print(f"Number of groups only in old approach:{len(set(old_groups) - set(assembly_groups))}")

Number of accessions only in new approach:20847
Number of accessions only in old approach:8
Number of groups only in new approach:35155
Number of groups only in old approach:97549


In [58]:
count = 0
for accession, item in old_grouped_accessions_to_group.items():
    if item == accession_to_true_group.get(accession, ''):
        count +=1
print(f"Number of accessions in old group same as truth: {count}")

count = 0
for accession, item in assembly_grouped_accessions_to_group.items():
    if item == accession_to_true_group.get(accession, ''):
        count +=1
print(f"Number of accessions in new group same as truth: {count}")

Number of accessions in old group same as truth: 666262
Number of accessions in new group same as truth: 809513


In [55]:
count = 0
count_same_as_truth = 0
for accession, item in old_grouped_accessions_to_group.items():
    if item == assembly_grouped_accessions_to_group.get(accession, ''):
        count +=1
        if item == accession_to_true_group.get(accession, ''):
            count_same_as_truth += 1
print(f"Number of accessions in the same group: {count}")
print(f"Number of accessions in the same group and in the same group as truth: {count_same_as_truth}")

Number of accessions in the same group: 986053
Number of accessions in the same group as truth: 666262


In [56]:
count = 0
for accession, item in old_grouped_accessions_to_group.items():
    assembly_group = assembly_grouped_accessions_to_group.get(accession, {})
    if item.issubset(assembly_group) and item != assembly_group:
        count +=(1/len(item))
print(f"Number of groups in old that are a strict subset of a group in new: {count}")

Number of groups in old that are a strict subset of a group in new: 97479.99999995595


In [57]:
count = 0
for accession, item in assembly_grouped_accessions_to_group.items():
    old_group = old_grouped_accessions_to_group.get(accession, {})
    if item.issubset(old_group) and item != old_group:
        count +=(1/len(item))
print(f"Number of groups in new that are a strict subset of a group in old: {count}")

Number of groups in new that are a strict subset of a group in old: 139.0
